# Missionars and Cannibals - a lab activity on search strategies
---

## Prerequisites

- Python 3 (correctly installed)

- module `numpy`

- the AIMA libraries for search; two alternatives:
    - either (git-)clone the AIMA repository: <https://github.com/aimacode/aima-python>
    - or download from the course site the `search.py` file

- file `reporting.py`: contains some useful functions for inspecting what's going on

- file `utils.py`: it is used by some search implementations, simply place it in the same folder.

---


## The AIMA search library

The library provided with the textbook contains an implementation of the most notable search strategies.

To use them, two main steps:
1. **Decide a representation of a state**. A state can be represented with any object you like. However, in order to correctly use the Graph-search algorithms, the `__eq__()` method should be overrided correctly. *Suggestion: in the first exercise, opt to represent the state using a tuple. The `__eq__()` method iis already properly overridden, so it would be an easy way for starting with.*

2. **Define a new class**, that will inherit form the class `Problem`, and that will contain the methods and the attributes needed to apply the search strategies. In particular, our new subclass of `Problem` will have to implement/override:
    - constructor `def __init__(self, initial, goal=None)`:
        
        > *'''The constructor specifies the initial state, and possibly a goal state, if there is a unique goal. Your subclass's constructor can add other arguments.'''*
    
    - method `def actions(self, state):`
        
        > *'''Return the actions that can be executed in the given state. The result would typically be a list, but if there are many actions, consider yielding them one at a time in an iterator, rather than building them all at once.'''*
        
    - method `def result(self, state, action):`
        
        > *'''Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state).'''*

    - method `def goal_test(self, state):`
        
        > *'''Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough.'''*
        
    - method `def path_cost(self, c, state1, action, state2):`
        
        > *'''Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1.'''*
        
    - method `def h(self, node):`
		
        > *'''Returns the heuristic applied to the Node node'''*

## The Missionars and Cannibals Problem

3 missionaries and 3 cannibals are on the same shore of a river, and want to cross such river. There is a single boat (on the same initial shore), able to bring around two persons at a time.
Should it happen that in ANY shore there are more cannibals than missionaries, the cannibals will eat the missionaries (failure states).

- Which state representation?
> Suggestion: a tuple of three numbers representing the number of missionaries, the number of cannibals, and the presence of the boat on the starting shore. 
> The initial state would be: `(3,3,1)`

- Which actions?
> The boat cross the river with passengers:
> - 1 missionary, 1 cannibal, 
> - 2 missionaries, 
> - 2 cannibals,  
> - 1 missionary 
> - 1 cannibal.

- Which is the goal?
> If the state is a tuple, the goal would be state `(0,0,0)`

- Which path cost?
> The number of river crossings might be a sensible choice.


In [6]:
from search import *
from reporting import *


class Missionars_Cannibals(Problem):

    def apply_move(self, state, action):
        m, c, boat = action
        miss, cann, boat_pos = state
    
        if boat_pos == 1:
            return (miss - m, cann - c, 0)
        else:
            return (miss + m, cann + c, 1)
    
    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        goal=(0,0,0) #come detto il goal è (0,0,0)
        # remember to invoke the constructor of the parent 
        super().__init__(initial, goal)

    # NOT NECESSARY, BUT...
    # suggestion: define a function that, given a state, returns True if the state is admissible
    def isValid(self, state):
        miss, cann, boat = state
        missR = 3 - miss
        cannR = 3 - cann
        # numero valido e non negativo
        if miss < 0 or cann < 0 or miss > 3 or cann > 3:
            return False
        # vincoli del gioco per non far mangiare i missionari
        if (miss > 0 and cann > miss) or (missR > 0 and cannR > missR):
            return False        
        return True

    def actions(self, state):
        possible_moves = [ (1,0), (2,0), (0,1), (0,2), (1,1) ]
        valid_actions = []
        for m, c in possible_moves:
            action = (m, c, state[2])
            new_state = self.apply_move(state, action)
            if self.isValid(new_state):
                valid_actions.append(action)
        return valid_actions

    def result(self, state, action):
        return self.apply_move(state, action)

    # if it is fine, leave it as it is
    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    # IMPLEMENT ONLY IF you want to experiment with uniform cost and informed strategies
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1
    
    # implement only if you want to experiment with informed strategies
    def h(self, node):
        """Returns the heuristic applied to the Node node""" 
        miss, cann, boat = node.state
        return (miss + cann - (1 if boat == 0 else 0)) // 2 # Stima minima delle traversate rimanenti, ignorando i vincoli tra missionari e cannibali.

######
      
initial_state=(3,3,1)
problem=Missionars_Cannibals(initial_state)

solution = breadth_first_tree_search(problem)

if solution:
    print("Solution found with cost:", solution.path_cost)
    
    path = path_states(solution)
    print(path)
    print()
else:
    print("No solution found.")


#####################

# to apply all the strategies, and print some report about used resources
# NOTICE: depth_first_tree_search is commented out... guess why? 
report([
    breadth_first_tree_search,
    breadth_first_graph_search,
    # depth_first_tree_search,
    depth_first_graph_search,
    astar_search
    ],
    [problem])

Solution found with cost: 11
[(3, 3, 1), (3, 1, 0), (3, 2, 1), (3, 0, 0), (3, 1, 1), (1, 1, 0), (2, 2, 1), (0, 2, 0), (0, 3, 1), (0, 1, 0), (1, 1, 1), (0, 0, 0)]

breadth_first_tree_search:
   26,706 inserted nodes |   11,753 tested goals |   11 cost |  11,763 expanded_nodes | Missionars_Cannibals
   26,706 inserted nodes |   11,753 tested goals |   11 cost |  11,763 expanded_nodes | TOTAL

breadth_first_graph_search:
       28 inserted nodes |       15 tested goals |   11 cost |      24 expanded_nodes | Missionars_Cannibals
       28 inserted nodes |       15 tested goals |   11 cost |      24 expanded_nodes | TOTAL

depth_first_graph_search:
       25 inserted nodes |       12 tested goals |   11 cost |      22 expanded_nodes | Missionars_Cannibals
       25 inserted nodes |       12 tested goals |   11 cost |      22 expanded_nodes | TOTAL

astar_search:
       28 inserted nodes |       14 tested goals |   11 cost |      24 expanded_nodes | Missionars_Cannibals
       28 inserted no

## 📊 Analisi del Report

### breadth_first_tree_search
- Molto inefficiente in termini di spazio e tempo.
- Esplora moltissimi stati ripetuti (perché non tiene traccia dei nodi già visitati).

### breadth_first_graph_search
- Molto più efficiente del tree search grazie al controllo sui nodi già visitati.
- Trova la soluzione ottimale con il minor numero di espansioni.
- È in generale una delle strategie più adatte per problemi con costo uniforme e spazio di stati limitato.

### depth_first_graph_search
- Buona efficienza, simile alla ricerca in ampiezza con grafo.
- Non garantisce di trovare la soluzione ottimale in generale, ma in questo caso ha trovato una soluzione con costo corretto (anche se per caso).
- Rischia di perdersi in rami profondi in problemi più complessi.

### astar_search
- Utilizza una funzione euristica semplice ma efficace.
- Stessa efficienza della breadth_first_graph_search in questo caso, segno che l’euristica è consistente ma non estremamente informativa.
- Ottimo bilanciamento tra completezza e prestazioni.

### depth_first_tree_search
Questa strategia è esclusa a priori (è commentata nel codice) perché non tiene traccia dei nodi visitati e, essendo una ricerca in profondità, può percorrere strade inutilmente lunghe, ripetendo le stesse mosse in loop, come ad esempio spostare avanti e indietro sempre gli stessi due elementi.